In [ ]:
import os
import sys
print("Python version" + sys.version)
os.getcwd()
print(sys.executable)

In [ ]:
import numpy as np
np.random.seed(123)
import pandas as pd
import scipy
import itertools

import umap
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import scanpy as sc
import anndata as ad
import scvelo as scv

from tqdm.notebook import tqdm

from pathlib import Path

TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
import cellrank as cr
from cellrank.kernels import CytoTRACEKernel

In [ ]:
cr.__version__

In [ ]:
import scanpy.external as sce

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score, homogeneity_completeness_v_measure, adjusted_mutual_info_score
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
sc.settings.verbosity = 1
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [ ]:
# remove weird grid from scvelo
plt.rcParams['axes.grid'] = False

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg', dpi = 300)

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = '8'

In [ ]:
# revised from Stefan's cell type signature
signatures_path_ = '../cell_type_from_stefan/scrnaseq_signature_collection/'
from score_and_classify import *

In [ ]:
# for using R plotting
%load_ext rpy2.ipython

In [ ]:
%%R

# R setting
suppressMessages(library('tidyverse'))
suppressMessages(library('grid'))
suppressMessages(library('ComplexUpset'))
suppressMessages(library('ggvenn'))
suppressMessages(library('lme4'))
suppressMessages(library('ggpubr'))


plot_theme_font_size = function(fontsize = 8){
    plot_theme =  theme(title = element_text(size = fontsize, family = "dajavu sans", colour = 'black', hjust = 0.5),
                   axis.title = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   axis.text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   legend.title =  element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   legend.text=element_text(size=fontsize, family = "dajavu sans", colour = 'black'),
                   text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   strip.text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   plot.margin = margin(0,0,0,0,"cm")) 
    plot_theme
}

In [ ]:
new_data_folder = '/fast/users/twei_m/work/crc/datasets_new_preprocessing'

In [ ]:
adata_all = sc.read(Path(new_data_folder)/'202401_adata_all_consensus_call_with_TC.h5')

In [ ]:
adata_epi = sc.read(Path(new_data_folder)/'202401_adata_epi_consensus_call_with_TC.h5')

In [ ]:
adata_all_noTC_cleanCI = sc.read(Path(new_data_folder)/'202401_adata_all_consensus_call_no_TC.h5')

In [ ]:
adata_epi_noTC_cleanCI = sc.read(Path(new_data_folder)/'202401_adata_epi_consensus_call_no_TC.h5')

In [ ]:
Uhlitz_scANVI_noTC_color_list_after = ['#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']
# no Tumour-like

In [ ]:
adata_epi.shape

In [ ]:
adata_epi[(adata_epi.obs['Uhlitz_scANVI_noTC'] != 'Tumour-like (normal_sample)')].shape

In [ ]:
39168-37670

### Fig4A

In [ ]:
fig = sc.pl.umap(adata_epi_noTC_cleanCI, 
                 color=['louvain'], 
               ncols=3, legend_loc='on data', size = 2, 
               frameon = False, return_fig=True)


### Fig4C

In [ ]:
ari_sample = adata_epi_noTC_cleanCI.obs['sample_origin'].copy()

In [ ]:
ari_ci = adata_epi_noTC_cleanCI.obs['cell_identity'].copy()
ari_ci = ari_ci.astype('str').replace('genomically_normal', 'normal_sample') # only 2 labels: GT and NN

In [ ]:
ari_icms = adata_epi_noTC_cleanCI.obs['iCMS_scANVI'].copy()
ari_icms = ari_icms.astype(str).replace('iCMS3\n(tumour sample)', 
                             'iCMS2\n(tumour sample)').replace('normal\n(tumour sample)',
                                                               'normal_sample')

In [ ]:
ari_infercnv = adata_epi_noTC_cleanCI.obs['inferCNV_result'].copy()
ari_infercnv = ari_infercnv.astype(str).replace('failed_sample', 
                             'CNA\n(tumour sample)').replace('CNN\n(tumour sample)',
                                                               'normal sample')

In [ ]:
ari_scitcem = adata_epi_noTC_cleanCI.obs['scitcem_call'].copy()
ari_scitcem = ari_scitcem.astype(str).replace('normal\n(tumour sample)', 
                             'normal_sample')

In [ ]:
ari_numbat = adata_epi_noTC_cleanCI.obs['numbat'].copy()
ari_numbat = ari_numbat.astype(str).replace('normal\n(tumour sample)', 
                             'normal_sample')

In [ ]:
ARI_res = [adjusted_rand_score(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
AMI_res = [adjusted_mutual_info_score(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
HCV_res = [homogeneity_completeness_v_measure(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]
#V is identical to normalized_mutual_info_score with the 'arithmetic' option for averaging.

In [ ]:
S_pca_res = [silhouette_score(X = adata_epi_noTC_cleanCI.obsm['X_pca'], labels = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
S_umap_res = [silhouette_score(X = adata_epi_noTC_cleanCI.obsm['X_umap'], labels = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
HCV_res_reshape = np.concatenate(HCV_res).reshape(6,3).transpose()

In [ ]:
#long format
cluster_eval_df = pd.DataFrame({'Label':['sample\norigin', 'inferCNV', 'iCMS','Scitcem', 'Numbat','consensus\ncall'],
             'Adjusted Rand Index': ARI_res,
             'Adjusted Mutual Information': AMI_res,
             'Homogeneity': HCV_res_reshape[0],
             'Completeness': HCV_res_reshape[1],
             'Silhouette (PCA)':S_pca_res,
             'Silhouette (UMAP)':S_umap_res}
             )

In [ ]:
cluster_eval_df_long = pd.melt(cluster_eval_df, 'Label', var_name='Metrics', value_name='Score')

In [ ]:
with plt.style.context('./plt_style'):
    plt.figure(figsize = (10,5))

    sns.barplot(data = cluster_eval_df_long[cluster_eval_df_long['Metrics'] == 'Homogeneity'], 
                      x = 'Label', y = 'Score',
                palette = ['#808080'])
    #'#f57379','#f5d107', '#228b22', '#47a2ed'
    plt.ylabel("Cluster homogeneity score")
    #sns.move_legend(ax, "upper right", bbox_to_anchor=(1.5, 1.0), 
    #                frameon=False)

;

### Fig4D

In [ ]:
cell_identity_list_clean = adata_epi_noTC_cleanCI.obs[[
    'cell_identity', 'sample_cap', 'sample_origin','patient','Uhlitz_scANVI', 'Uhlitz_scANVI_noTC', 'louvain']]

In [ ]:
%%R -i cell_identity_list_clean,Uhlitz_scANVI_noTC_color_list_after -w 1000 -h 500 -r 200

# cell type differences in genomically normal vs normal samples

df = cell_identity_list_clean %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
filter(louvain == '9') %>%
group_by(cell_identity, Uhlitz_scANVI_noTC) %>%
mutate(sum=n()) %>%
distinct(cell_identity, Uhlitz_scANVI_noTC, sum) 


df$Uhlitz_scANVI_noTC = factor(df$Uhlitz_scANVI_noTC, rev(levels(df$Uhlitz_scANVI_noTC)))

p = df %>% 
ggplot(aes(x = sum, y = cell_identity, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill') +
theme_void() + 
scale_fill_manual(values = rev(Uhlitz_scANVI_noTC_color_list_after), name = 'Epithelial\ncell type') +
labs(x = '\nProportion\n', 
     y = '') +
guides(fill=guide_legend(nrow=4)) +
scale_x_continuous(n.breaks = 3) +
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
      legend.justification = c(1.5,0),
      legend.position = 'bottom',
      legend.direction='horizontal') +
scale_y_discrete(labels = rev(c('normal\nsample (15)', 'genomically\nnormal (1281)')))

print(p)

ggsave(file="GN_NN_cell_type_bar_c9.svg", width = 10, height = 8, units = "cm", dpi = 300)

In [ ]:
df=adata_epi.obs[(adata_epi.obs['cell_identity']=='genomically_normal') & (adata_epi.obs['louvain']=='9')].groupby('Uhlitz_scANVI_noTC').size().drop('Tumour-like (normal_sample)',axis=0).reset_index()
colors=pd.Series(adata_epi.uns['Uhlitz_scANVI_noTC_colors'],index=adata_epi.obs['Uhlitz_scANVI_noTC'].cat.categories)


In [ ]:
fig=plt.figure(figsize=(4,4))
plt.pie(df[0], labels=df['Uhlitz_scANVI_noTC'], colors=colors.loc[df['Uhlitz_scANVI_noTC']]);

### Fig4E cluster 9 markers

In [ ]:
plt.figure(figsize = (2,1))
sc.pl.dotplot(adata_epi_noTC_cleanCI, ['PLA2G2A', 'PIGR', 'LEFTY1', 
                                                            'B3GNT7', 'MUC12','LCN2', 
                                                            'OLFM4', 'PI3', 'SLPI', 
                                                            'GOLM1','NUPR1', 'MUC1',
                                                           'MUC5B', 'SELENOP', 'ZG16'],
              groupby=['louvain'], dendrogram=False, swap_axes=True);


### Fig5B

In [ ]:
order_start_with_tumour_noTC = [
 'Tumour-like',
 'Stem',
 'Stem/TA',
 'Immature Goblet',
 'Goblet',
 'Enterocyte progenitor',
 'Enterocytes',
 'Tuft']

In [ ]:
sample_color_map = dict(zip(adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'].cat.categories, 
                            adata_epi_noTC_cleanCI.uns['Uhlitz_scANVI_noTC_colors']))

In [ ]:
plt.figure(figsize = (2,2))

g = sns.JointGrid(data = adata_epi_noTC_cleanCI.obs, x = 'DC1', y = 'DC2', hue = 'Uhlitz_scANVI_noTC',
                      hue_order=order_start_with_tumour_noTC,
                      palette=[sample_color_map[key] for key in order_start_with_tumour_noTC],
                      height = 6, ratio = 3)
g.plot_marginals(sns.histplot, kde = False, binwidth=0.001, edgecolor = 'black', linewidth = 0.3, alpha =1)

#sns.move_legend(g.ax_joint, "upper left", bbox_to_anchor=(1.5, 1), title = 'Epithelial cell type', 
#                    frameon=False, title_fontsize = 8)

#g.ax_joint.legend_.remove()
;

In [ ]:
fig = sc.pl.diffmap(adata_epi_noTC_cleanCI, color=['Uhlitz_scANVI_noTC'], 
                    size = 2, frameon = True, return_fig=True, 
                    title = '', legend_loc = 'None')

In [ ]:
#consume too many MB 
plt.figure(figsize = (5/2.54,5/2.54))

g = sns.JointGrid(data = adata_epi_noTC_cleanCI.obs, x = 'DC1', y = 'DC2', hue = 'Uhlitz_scANVI_noTC',
                      hue_order=order_start_with_tumour_noTC,
                      palette=[sample_color_map[key] for key in order_start_with_tumour_noTC],
                      height = 9, ratio = 3)
g.plot_joint(sns.scatterplot, s = 2, edgecolor = None)
g.plot_marginals(sns.histplot, kde = False, binwidth=0.001, edgecolor = 'black', linewidth = 0.3, alpha =1)

sns.move_legend(g.ax_joint, "upper left", bbox_to_anchor=(1.5, 1), title = 'Epithelial cell type', 
                    frameon=False, title_fontsize = 8)

g.ax_joint.legend_.remove()
;

### Fig5D

In [ ]:
fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['ct_pseudotime'], 
               ncols=1, legend_loc='right margin', size = 2, title = 'Cytotrace pseudotime',
               frameon = False, return_fig=True, color_map='viridis_r')


### SupFig3E neighbor

In [ ]:
n_neighbors=30
aa=scipy.sparse.csr_matrix((adata_epi.obs['patient'].iloc[adata_epi.obsp['distances'].nonzero()[1]].values==adata_epi.obs['patient'].iloc[adata_epi.obsp['distances'].nonzero()[0]].values, 
                            adata_epi.obsp['distances'].nonzero()))
aa.data[np.isnan(aa.data)]=0
aa.eliminate_zeros()
adata_epi.obs['frac_neighbors_same_patient']=aa.sum(1).A1/n_neighbors

In [ ]:
df=adata_epi.obs.groupby(['cell_identity','patient']).agg({'frac_neighbors_same_patient': 'mean'}
                                                  ).drop('no confident assignment',axis=0,level=0)
df.index=pd.MultiIndex.from_tuples(df.index.tolist(),names=['cell_identity','patient'])

In [ ]:
plt.figure(figsize = (5,4))

sns.boxplot(data=df.reset_index(),x='cell_identity',y='frac_neighbors_same_patient', showfliers=False,
           palette = ['#ff7f0e','#1f77b4','#808080'])
sns.swarmplot(data=df.reset_index(), x='cell_identity',y='frac_neighbors_same_patient',hue='patient')
plt.legend(bbox_to_anchor=(1.1,1))
;


### SupFig5

In [ ]:
CRC_sig = pd.read_excel("../datasets_new_preprocessing/Signatures_Single_cells.xlsx", 
                        skiprows=[1], index_col=None, sheet_name = 'fixed_var_name')
for i in CRC_sig.columns:
    score_genes(adata_epi_noTC_cleanCI, np.array(CRC_sig[i].dropna()), score_name=i)

In [ ]:
20455/5623*7

In [ ]:
adata_epi_noTC_cleanCI.obs['cell_identity'].value_counts()

In [ ]:
ax2 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='genomically_normal')], 
                        CRC_sig.columns, groupby='Uhlitz_scANVI_noTC', 
                        dendrogram=False, swap_axes=True, figsize=[7/2.54,7/2.54], 
                        vcenter= 0.5,
                        cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#c46868,#c74242,#CD0000", as_cmap=True),
                        show = False,
                        vmax = 1.5, vmin = -0.5
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
ax2['heatmap_ax'].set_title('genomically normal', fontsize = 8)
ax2['groupby_ax'].set_xlabel('', fontsize = 8)
ax2['heatmap_ax'].tick_params(labelsize=8) 

;

    #ax2['groupby_ax'].plot(c = ['red']*7)
    #print(ax2)
#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
ax3 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='normal_sample')], 
                  CRC_sig.columns, groupby='Uhlitz_scANVI_noTC', 
                   dendrogram=False, swap_axes=True, figsize=[7/2.54,7/2.54], 
                  vcenter= 0.5,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#c46868,#c74242,#CD0000", as_cmap=True),
                    show = False,
                   vmax = 1.5, vmin = -0.5
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
ax3['heatmap_ax'].set_title('normal sample', fontsize = 8)
ax3['groupby_ax'].set_xlabel('', fontsize = 8)
ax3['heatmap_ax'].tick_params(labelsize=8) 
;
#AttributeError: 'AxesImage' object has no property 'row_colors'

### SupFig5A

In [ ]:
cell_identity_list = adata_epi.obs[[
    'cell_identity', 'sample_cap', 'sample_origin','patient','Uhlitz_scANVI', 'Uhlitz_scANVI_noTC']]

In [ ]:
cell_type_list = adata_epi.obs['Uhlitz_scANVI'].cat.categories.values

In [ ]:
%%R -i cell_identity_list,cell_type_list -w 1400 -h 800 -r 200

# Fig 4c
# Statistical testing of cell type differences in genomically normal vs normal samples

df = cell_identity_list %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
filter(!patient %in% c('P08', 'P21', 'P26', 'P35') ) 

df$cell_identity = factor(df$cell_identity, levels = c('normal_sample', 'genomically_normal'))

summary_model = list()
summary_p = list()

for(i in cell_type_list){
    glmer_model = glmer(Uhlitz_scANVI == i ~ cell_identity + (1|patient), 
             family="binomial", data = df)
    summary_model[[i]] = summary(glmer_model)
    summary_p[[i]] = data.frame('Uhlitz_scANVI' = i, 
           'p' = summary(glmer_model)$coefficients['cell_identitygenomically_normal','Pr(>|z|)'])
}
# boundary (singular) fit: see help('isSingular')
# means random effects are very small

glmer_p_stats = bind_rows(summary_p) %>% 
filter(!Uhlitz_scANVI %in% c('TC1','TC2','TC3','TC4')) %>%
mutate(loc = seq(1,7,1), p = signif(p,2)) %>%
mutate(star = case_when(p < 0.05 ~ '< 0.05'))

#print(glmer_p_stats)

In [ ]:
%%R -i cell_identity_list,cell_type_list -w 1400 -h 800 -r 200

g = table(cell_identity_list[c('Uhlitz_scANVI', 'patient', 'cell_identity')]) %>%
as.data.frame() %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
mutate(cell_identity = factor(cell_identity, levels = c('normal_sample','genomically_normal'))) %>%
filter(!patient %in% c('P08', 'P21', 'P26', 'P35') ) %>%
filter(!Uhlitz_scANVI %in% c('TC1','TC2','TC3','TC4')) %>%
group_by(cell_identity, patient) %>% 
mutate(frac_sample = Freq/sum(Freq)) %>% 
ggplot(aes(x = Uhlitz_scANVI, y = frac_sample)) +
plot_theme_font_size(8) +
geom_boxplot(outlier.color = 'white', aes(color = cell_identity)) +
geom_point(position=position_jitterdodge(), aes(color = cell_identity), size = 1)+
theme_bw() + 
theme(axis.text.x = element_text(angle = 90),
      panel.grid = element_blank()) +
labs(x = 'Epithelial cell types',
     y = 'Fraction per sample\n') +
scale_color_manual(values = rev(c('#1f77b4', '#808080')), 
                   name = 'Consensus call',
                   labels = c('normal\nsample', 'genomically\nnormal')) +
geom_bracket(aes(xmin = loc-0.25, xmax = loc+0.25, label = p), 
             data = glmer_p_stats, 
             y.position = c(0.45, 0.4, 0.2, 0.4, 0.45, 0.55, 0.25), size = 0.4) +
ylim(0,0.65) 

print(g)

ggsave(file="cell_composition_test.svg", width = 15, height = 9 , units = "cm", dpi = 300)

In [ ]:
all_cell_list = adata_all_noTC_cleanCI.obs[['patient', 'sample_cap',
                                            'celltype_1a','cell_type_level2', 'sample_origin']]

In [ ]:
removed_TC_id = list(set(adata_epi.obs.index) - set(adata_epi_noTC_cleanCI.obs.index))

In [ ]:
%%R -i cell_identity_list_clean,all_cell_list,removed_TC_id -w 1000 -h 500 -r 200

# Fig. 5a 
# proportion of each cell compartment (epi, imm, str) of each sample
suppressMessages(library(tidyverse))


df = all_cell_list %>%
merge(cell_identity_list_clean[c('cell_identity', 'Uhlitz_scANVI_noTC')], by = 0, all.x = TRUE) %>%
#filter(Row.names %notin% removed_TC_id) %>%
mutate(cell_type_merge = coalesce(cell_identity, celltype_1a)) %>%
group_by(sample_cap, cell_type_merge) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap,cell_type_merge, sum, sum_sample, patient, sample_origin)

df$cell_type_merge = factor(df$cell_type_merge, levels = c('imm', 'str', 'normal_sample',
                                                             'genomically_normal', 
                                                             'genomically_tumour'))

p = df %>%
ggplot(aes(x = sum, y = patient, fill = cell_type_merge)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ED5D47',
                            '#4EAC57',
                             '#144FAC',
                            '#1f77b4',
                            '#ff7f0e'),
                   name = 'Cell identity',
                   labels = c('Immune',
                             'Stromal', 
                              'Epithelial',
                             'Genomically\nnormal',
                             'Genomically\ntumour')) +
theme_void() + 
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
      panel.spacing = unit(10, "pt")) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$patient)))) +
facet_grid(cols = vars(sample_origin), 
           labeller = as_labeller(c('normal' = 'Normal\nsample', 'tumour' = 'Tumour\nsample')))
print(p)

ggsave(file="cell_identity_and_TME_normal_tumour_sample.svg", width = 8, height = 5, units = "cm", dpi = 300)


In [ ]:
sample_color_map_for_all = sample_color_map.copy()

sample_color_map_for_all.update(dict(zip(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories, 
         ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
          '#CE9486', '#8f7022', '#cf7708', '#B5B867',
          '#DEB058', '#899DA4','#1aa375', '#AA82A7', '#c44545'
         ])))

In [ ]:
sample_color_map_grey = dict(zip(adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'].cat.categories, 
                            ['#d3d3d3']*8))

In [ ]:
sample_color_map_grey.update(dict(zip(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories, 
         ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
          '#CE9486', '#8f7022', '#cf7708', 
          '#B5B867',
          '#DEB058', '#899DA4','#1aa375', '#AA82A7', '#c44545'
         ])))

In [ ]:
sample_color_map_grey.update(dict(zip(['TC1', 'TC2', 'TC3', 'TC4'], ['#d3d3d3']*8)))

In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'], 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='lower center', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[(adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi') & 
                        (adata_all_noTC_cleanCI.obs['sample_origin'] == 'tumour')], 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[(adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi') & 
                        (adata_all_noTC_cleanCI.obs['sample_origin'] != 'tumour')], 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_for_all)
